# SAM 3D Objects - 單張圖片轉完整 3D 模型

**使用官方 Meta SAM 3D Objects**

**使用說明**：
1. 點擊選單 **「執行階段 → 變更執行階段類型」**，選擇 **「T4 GPU」**
2. 依序執行每個 Cell（按 `Shift + Enter`）
3. 第一次執行需要輸入 Hugging Face Token

---

## 步驟 1: 安裝 SAM 3D Objects

In [ ]:
%%capture
# 克隆官方 SAM 3D Objects 倉庫
!git clone https://github.com/facebookresearch/sam-3d-objects.git
%cd sam-3d-objects

# 安裝 PyTorch (CUDA 版本)
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# 安裝其他依賴
!pip install huggingface-hub transformers diffusers accelerate
!pip install trimesh plyfile imageio opencv-python pillow
!pip install omegaconf hydra-core

print("✅ SAM 3D Objects 安裝完成！")

## 步驟 2: 登入 Hugging Face

**首次使用需要**：
1. 前往 [Hugging Face Tokens](https://huggingface.co/settings/tokens) 建立 Token
2. 前往 [SAM 3D Objects](https://huggingface.co/facebook/sam-3d-objects) 申請存取權限
3. 等待批准（通常幾分鐘內）
4. 將 Token 貼到下方執行

In [ ]:
from huggingface_hub import login

# 輸入您的 Hugging Face Token
token = input("請輸入 Hugging Face Token: ")
login(token=token)

print("✅ 已成功登入 Hugging Face")

## 步驟 3: 下載模型權重

In [ ]:
from huggingface_hub import snapshot_download
import os

# 下載 SAM 3D Objects 模型
print("📥 開始下載模型權重（約 2-3 分鐘）...")

model_path = snapshot_download(
    repo_id="facebook/sam-3d-objects",
    local_dir="checkpoints/hf",
    local_dir_use_symlinks=False
)

print(f"✅ 模型已下載到: {model_path}")

## 步驟 4: 上傳您的圖片

In [ ]:
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt

# 上傳圖片
print("📤 請選擇您要轉換的圖片（支援 JPG, PNG）...")
uploaded = files.upload()

# 取得檔案名稱
filename = list(uploaded.keys())[0]

# 預覽圖片
img = Image.open(filename)
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.title(f'上傳的圖片: {filename}')
plt.show()

print(f"✅ 圖片尺寸: {img.size}")
print(f"✅ 已上傳: {filename}")

## 步驟 5: 載入 SAM 3D 模型

In [ ]:
import sys
import os

# 設定環境變數
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['CONDA_PREFIX'] = '/usr/local'

# 添加路徑
sys.path.insert(0, '/content/sam-3d-objects')
sys.path.insert(0, '/content/sam-3d-objects/notebook')

# 載入模型
from inference import Inference, load_image

print("🔄 載入 SAM 3D Objects 模型...")
config_path = "/content/sam-3d-objects/checkpoints/hf/pipeline.yaml"
inference = Inference(config_path, compile=False)

print("✅ 模型載入完成！")

## 步驟 6: 生成 3D 模型

In [ ]:
import time

# 載入圖片
image = load_image(filename)

# 生成 3D 模型
print("🎨 開始生成 3D 模型...")
print("⏱️  預計需要 20-40 秒，請稍候...")
start_time = time.time()

output = inference(image, seed=42)

elapsed_time = time.time() - start_time
print(f"✅ 3D 模型生成完成！耗時: {elapsed_time:.2f} 秒")

# 儲存為 .ply 格式
output_filename = "output.ply"
output["gs"].save_ply(output_filename)

print(f"💾 已儲存: {output_filename}")
print(f"📊 檔案大小: {os.path.getsize(output_filename) / (1024*1024):.2f} MB")

## 步驟 7: 下載 3D 模型

In [ ]:
from google.colab import files

# 下載 .ply 檔案
print("⬇️ 開始下載...")
files.download(output_filename)

print(f"✅ 已下載: {output_filename}")
print("")
print("📌 如何查看 3D 模型：")
print("1. 線上查看器: https://playcanvas.com/supersplat/editor")
print("2. 本地 Web UI: http://localhost:3000")
print("   - 將 .ply 檔案放到專案的 web-ui/outputs/ 目錄")
print("   - 重新整理網頁即可查看")
print("3. 桌面軟體: Blender, MeshLab")

---

## 🎉 完成！

您現在擁有一個完整的 360° 3D 模型，包含：
- ✅ 完整的背面幾何結構
- ✅ 高品質的紋理
- ✅ 可以從任意角度查看

### 可選：轉換為 OBJ 格式

In [ ]:
# 如果您需要 .obj 格式（更通用）
!pip install -q pymeshlab

import pymeshlab as ml

try:
    ms = ml.MeshSet()
    ms.load_new_mesh('output.ply')
    ms.save_current_mesh('output.obj')
    
    files.download('output.obj')
    print("✅ 已下載: output.obj")
except Exception as e:
    print(f"⚠️ OBJ 轉換失敗: {str(e)}")
    print("建議直接使用 .ply 檔案")